In [81]:
import pandas as pd
import sqlite3
from datetime import datetime

df = pd.read_csv('/Users/shreyansh/Downloads/Traffic_Incident_Calls_db.csv')

df = df.rename(columns={
    'Complaint ID': 'complaint_id',
    'Accident Type': 'accident_type',
    'Report Date': 'report_date',
    'Address': 'address',
    'ZIP CODE': 'zip_code',
    'Council District 2011': 'council_district',
    'POLICE DISTRICT': 'police_district',
    'Neighborhood': 'neighborhood'
})

keep_cols = ['complaint_id', 'accident_type', 'report_date', 'address', 'zip_code', 
             'council_district', 'police_district', 'neighborhood']
df_clean = df.loc[:, keep_cols].copy()  
df_clean['report_date'] = pd.to_datetime(df_clean['report_date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
df_clean['report_date_only'] = df_clean['report_date'].dt.date
df_clean['report_time_only'] = df_clean['report_date'].dt.time
df_clean = df_clean.drop(columns=['report_date'])


print(df_clean.head())

df_clean.to_csv('traffic_incidents_cleaned.csv', index=True)



   complaint_id    accident_type            address zip_code council_district  \
0      18449059  ACCIDENT/INJURY    UNITY ISLAND DR    14220          LOVEJOY   
1      18715587  ACCIDENT/INJURY      341 SENECA ST    14204         ELLICOTT   
2      18637960  ACCIDENT/INJURY      444 FOREST AV    14213            NORTH   
3       1548888  ACCIDENT/INJURY  380 N OAK ,BUF ST    14203         ELLICOTT   
4       1212110  ACCIDENT/INJURY         203 BOX AV    14211         FILLMORE   

  police_district     neighborhood report_date_only report_time_only  
0      District A    Hopkins-Tifft       2021-09-02         18:10:10  
1      District A         Ellicott       2022-01-21         13:18:00  
2      District D  Elmwood Bidwell       2021-12-08         11:54:02  
3      District B          Central       2006-11-20         06:14:28  
4      District C         MLK Park       2006-01-29         22:18:20  


In [82]:
import sqlite3

conn = sqlite3.connect("/Users/shreyansh/Downloads/Traffic_Incident_Calls_db.db")
cursor = conn.cursor()


In [83]:

cursor.executescript("""
CREATE TABLE IF NOT EXISTS ZipCodes (
    zip_code TEXT PRIMARY KEY
);

CREATE TABLE IF NOT EXISTS CouncilDistricts (
    council_district_id INTEGER PRIMARY KEY AUTOINCREMENT,
    district_number TEXT UNIQUE
);


CREATE TABLE IF NOT EXISTS PoliceDistricts (
    police_district_id INTEGER PRIMARY KEY AUTOINCREMENT,
    district_name TEXT UNIQUE
);

CREATE TABLE IF NOT EXISTS Neighborhoods (
    neighborhood_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
);

CREATE TABLE IF NOT EXISTS Locations (
    location_id INTEGER PRIMARY KEY AUTOINCREMENT,
    address TEXT,
    zip_code TEXT,
    council_district_id INTEGER,
    police_district_id INTEGER,
    neighborhood_id INTEGER,
    FOREIGN KEY (zip_code) REFERENCES ZipCodes(zip_code),
    FOREIGN KEY (council_district_id) REFERENCES CouncilDistricts(council_district_id),
    FOREIGN KEY (police_district_id) REFERENCES PoliceDistricts(police_district_id),
    FOREIGN KEY (neighborhood_id) REFERENCES Neighborhoods(neighborhood_id)
);

CREATE TABLE IF NOT EXISTS Incidents (
    complaint_id TEXT PRIMARY KEY,
    accident_type TEXT,
    report_date DATE,
    report_time TIME,
    location_id INTEGER,
    FOREIGN KEY (location_id) REFERENCES Locations(location_id)
);
""")



In [84]:

for zip_code in df_clean['zip_code'].dropna().unique():
    cursor.execute("INSERT OR IGNORE INTO ZipCodes (zip_code) VALUES (?)", (zip_code,))

for district in df_clean['council_district'].dropna().unique():
    cursor.execute("INSERT OR IGNORE INTO CouncilDistricts (district_number) VALUES (?)", (district,))


for pd in df_clean['police_district'].dropna().unique():
    cursor.execute("INSERT OR IGNORE INTO PoliceDistricts (district_name) VALUES (?)", (pd,))

for n in df_clean['neighborhood'].dropna().unique():
    cursor.execute("INSERT OR IGNORE INTO Neighborhoods (name) VALUES (?)", (n,))
    
conn.commit()


OperationalError: database is locked

In [ ]:
for _, row in df_clean.iterrows():

    cursor.execute("""
    INSERT INTO Locations (address, zip_code, council_district_id, police_district_id, neighborhood_id)
    VALUES (
        ?, ?, 
        (SELECT council_district_id FROM CouncilDistricts WHERE district_number = ?),
        (SELECT police_district_id FROM PoliceDistricts WHERE district_name = ?),
        (SELECT neighborhood_id FROM Neighborhoods WHERE name = ?)
    )
    """, (row['address'], row['zip_code'], row['council_district'], row['police_district'], row['neighborhood']))
    
    location_id = cursor.lastrowid

    report_time_val = row['report_time_only']
    report_time_str = (
        report_time_val.strftime('%H:%M:%S') if hasattr(report_time_val, 'strftime')
        else report_time_val if isinstance(report_time_val, str)
        else None
    )

    cursor.execute("""
    INSERT OR IGNORE INTO Incidents (complaint_id, accident_type, report_date, report_time, location_id)
    VALUES (?, ?, ?, ?, ?)
    """, (
        row['complaint_id'],
        row['accident_type'],
        str(row['report_date_only']) if pd.notnull(row['report_date_only']) else None,
        report_time_str,
        location_id
    ))

conn.commit()



In [ ]:
# checking 
cursor.execute("""
SELECT i.complaint_id, i.accident_type, i.report_date, i.report_time,
       l.address, z.zip_code, c.district_number, p.district_name, n.name
FROM Incidents i
JOIN Locations l ON i.location_id = l.location_id
JOIN ZipCodes z ON l.zip_code = z.zip_code
JOIN CouncilDistricts c ON l.council_district_id = c.council_district_id
JOIN PoliceDistricts p ON l.police_district_id = p.police_district_id
JOIN Neighborhoods n ON l.neighborhood_id = n.neighborhood_id
LIMIT 5;
""")

for row in cursor.fetchall():
    print(row)


In [85]:
query = """
SELECT
  strftime('%H', report_time) AS hour_of_day,
  COUNT(*) AS incident_count
FROM incidents
GROUP BY hour_of_day
ORDER BY incident_count DESC;
"""
df_time_of_day = pd.read_sql_query(query, conn)
df_time_of_day

,hour_of_day,incident_count
0,15,18689
1,16,18483
2,17,17789
3,14,15366
4,13,13548
5,12,13521
6,18,12868
7,11,11333
8,08,10779
9,19,9935


In [86]:
query_2 = """
SELECT
    cd.district_number AS council_district,
    pd.district_name AS police_district,
    l.zip_code,
    COUNT(*) AS total_incidents
FROM Incidents i
JOIN Locations l ON i.location_id = l.location_id
JOIN CouncilDistricts cd ON l.council_district_id = cd.council_district_id
JOIN PoliceDistricts pd ON l.police_district_id = pd.police_district_id
GROUP BY council_district, police_district, zip_code
ORDER BY total_incidents DESC
LIMIT 10;
"""

df_query_2 = pd.read_sql_query(query_2, conn)
df_query_2

,council_district,police_district,zip_code,total_incidents
0,UNIVERSITY,District E,14215,17226
1,NORTH,District D,14207,14383
2,DELAWARE,District D,14216,12285
3,ELLICOTT,District B,14203,8736
4,SOUTH,District A,14220,8318
5,MASTEN,District E,14215,8304
6,FILLMORE,District C,14211,6500
7,DELAWARE,District D,14214,5790
8,NORTH,District D,14216,5691
9,LOVEJOY,District C,14211,5343


In [87]:
query_3 = """
SELECT
    l.address,
    COUNT(*) AS incident_count
FROM Incidents i
JOIN Locations l ON i.location_id = l.location_id
GROUP BY l.address
HAVING incident_count > 500
ORDER BY incident_count DESC
LIMIT 10;
"""

df_query_3 = pd.read_sql_query(query_3, conn)
df_query_3


,address,incident_count
0,SCAJAQUADA EB EX @ DELAWARE AV,1027
1,HERTEL AV @ DELAWARE AV,854
2,CLINTON ST @ BAILEY AV,803
3,KENSINGTON EB EX @E DELAVAN AV,737
4,SCAJAQUADA EB EX @ KENSINGTON RAMP EX,667
5,SKYWAY INBOUND,665
6,SCAJAQUADA WB EX @ KENSINGTON RAMP EX,654
7,WALDEN AV @ BAILEY AV,650
8,MAIN ST @ HUMBOLDT PW,648
9,HERTEL AV @ ELMWOOD AV,637


In [88]:
query_4 = """
SELECT
    strftime('%Y', report_date) AS year,
    COUNT(*) AS total_incidents
FROM Incidents
GROUP BY year
ORDER BY year ASC;
"""

df_query_4 = pd.read_sql_query(query_4, conn)
df_query_4


,year,total_incidents
0,2006,10506
1,2007,11825
2,2008,11875
3,2009,11330
4,2010,11504
5,2011,11197
6,2012,10153
7,2013,10838
8,2014,11346
9,2015,12382


In [89]:
query_5 = """
SELECT
    accident_type,
    COUNT(*) AS total_incidents
FROM Incidents
GROUP BY accident_type
ORDER BY total_incidents DESC;
"""

df_query_5 = pd.read_sql_query(query_5, conn)
df_query_5


,accident_type,total_incidents
0,ACCIDENT PROPERTY DAMAGE ONLY,161528
1,ACCIDENT/INJURY,44061
2,ACCIDENT SKYWAY/33/198,11719


In [90]:
query_accident_types_by_year = """
SELECT
    strftime('%Y', report_date) AS year,
    accident_type,
    COUNT(*) AS total_incidents
FROM Incidents
GROUP BY year, accident_type
ORDER BY year ASC, total_incidents DESC;
"""

df_accident_types_by_year = pd.read_sql_query(query_accident_types_by_year, conn)
df_accident_types_by_year


,year,accident_type,total_incidents
0,2006,ACCIDENT PROPERTY DAMAGE ONLY,8108
1,2006,ACCIDENT/INJURY,2398
2,2007,ACCIDENT PROPERTY DAMAGE ONLY,8792
3,2007,ACCIDENT/INJURY,2443
4,2007,ACCIDENT SKYWAY/33/198,590
5,2008,ACCIDENT PROPERTY DAMAGE ONLY,8699
6,2008,ACCIDENT/INJURY,2521
7,2008,ACCIDENT SKYWAY/33/198,655
8,2009,ACCIDENT PROPERTY DAMAGE ONLY,8222
9,2009,ACCIDENT/INJURY,2492


In [91]:
conn.commit()
cursor.close()
conn.close()